In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os

import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, Sequential
from keras.applications.resnet50 import ResNet50, preprocess_input as resnet50_preprocess_input
#from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess_input
#from keras.applications.xception import Xception, preprocess_input as xception_preprocess_input
#from keras.applications.vgg19 import VGG19, preprocess_input as vgg19_preprocess_input
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.utils.np_utils import to_categorical
from keras import Model

from constants import img_size, source_dir_dev, source_dir_train, source_dir_dev_gray, source_dir_train_gray

Using TensorFlow backend.


In [2]:
def generator(datagen, data_dir, batch_size):
    print('Data directory:', data_dir)
    return datagen.flow_from_directory(
        data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=0,
    )

In [3]:
def base_model(model_constructor, name, preprocessor, pooling=None, verbose=True, layer=None):    
    base_model = model_constructor(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3), pooling=pooling)
    
    model = None
    if layer == None:
        model = base_model
    else:
        output = base_model.layers[layer].output
        
        if pooling != None:
            output_shape = base_model.layers[layer].output_shape
            Pooling = AveragePooling2D if pooling == 'avg' else MaxPooling2D
            print(output_shape)
            output = Pooling(pool_size=(output_shape[1], output_shape[2]))(output)
            
        model = Model(input=base_model.input, output=output)
    
    if verbose == True:
        model.summary()
        
    return model

In [4]:
def paths(name):
    paths_to_create = [
        './data/features/' + name,
    ]
    
    for path in paths_to_create:
        if not os.path.exists(path):
            os.makedirs(path)
            
    return paths_to_create[0]

In [5]:
def extractor(model_constructor, name, preprocessor, batch_size, pooling=None, layer=None, verbose = True):
    model = base_model(model_constructor, name, preprocessor, pooling, verbose, layer)

    datagen = ImageDataGenerator(preprocessing_function=preprocessor)
    generator_train = generator(datagen, source_dir_train_gray, batch_size)
    generator_dev = generator(datagen, source_dir_dev_gray, batch_size)
    
    path = paths(name)
    print('Path:', path)
    np.save(path + '/filenames_train.npy', generator_train.filenames)
    np.save(path + '/classes_train.npy', generator_train.classes)
    np.save(path + '/filenames_dev.npy', generator_dev.filenames)
    np.save(path + '/classes_dev.npy', generator_dev.classes)
    
    #train features
    features_train = model.predict_generator(generator_train, len(generator_train), verbose=1)
    np.save(path + '/features_train.npy', features_train)
    
    #dev features
    features_dev = model.predict_generator(generator_dev, len(generator_dev), verbose=1)
    np.save(path + '/features_dev.npy', features_dev)

In [6]:
extractor(
    model_constructor=ResNet50, 
    name='res_net_gray', 
    preprocessor=resnet50_preprocess_input, 
    batch_size=8, 
#    layer=16,
    pooling='avg'
)

94658560/94653016 [==============================] - 22s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________

Found 7227 images belonging to 196 classes.
Data directory: data/dev_dataset_gray
Found 917 images belonging to 196 classes.
Path: ./data/features/res_net_gray
115/115 [==============================] - 796s 7s/step
